In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
# 모델 및 토크나이저 로드
model_path = "./model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# 요약할 텍스트
text = """
엔비디아가 차세대 게이밍 GPU(그래픽처리장치) '지포스 RTX 50시리즈'를 내놓으며 PC용 AI칩 시장을 겨냥한다.
올해 초 미국 라스베이거스서 개최된 CES 2025에서 선보인 바 있는 RTX 50 시리즈는 개인용 AI PC 시장을 바꿔놓을 게임 체인저로 지목되면서 향후 인텔, 퀄컴, AMD 등과의 치열한 각축전이 예상되고 있다.
20일 엔비디아는 서울 코엑스에서 'RTX AI PC 데이'를 열고 새로운 아키텍처 '블랙웰'이 탑재된 '지포스 RTX 50 시리즈'를 발표했다.
이는 RTX는 엔비디아가 만든 GPU(그래픽처리장치)인데 이번에 새롭게 나온 50 시리즈(5090, 5080, 5070TI, 5070)등은 이전 세대보다 강화된 AI(인공지능) 기술과 낮아진 가격을 자랑한다.
이중에서도 5090 제품의 경우 최고급 PC에 주로 사용되며 게이밍, 그래픽 연산 등에 주로 사용된다. 게이머 뿐 아니라 개발자, 크리에이터 등을 위한 최고 사양을 자랑한다.
RTX 5090의 경우 초당 데이터 전송량이 초당 1.8TB(테라바이트)로, 이전 모델인 RTX 4090보다 두 배 향상됐다.
성능은 높아졌으나 배터리 효율은 더욱 개선됐다는 것이 엔비디아 측의 설명이다.
"""

# 요약 지시를 포함한 입력 생성
instruction = "위 텍스트를 간결하게 요약해 주세요."
messages = [
    {"role": "user", "content": f"{instruction}\n\n{text}"}
]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

In [4]:
# 텍스트 생성
outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


In [5]:
# 생성된 텍스트 디코딩 및 출력
summary = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
print("요약 결과:")
print(summary)

요약 결과:
엔비디아가 '지포스 RTX 50 시리즈'를 발표하며 PC용 AI칩 시장을 겨냥한다. 이 시리즈는 '블랙웰' 아키텍처를 탑재한 새로운 GPU로, AI 기술과 가격이 강화된 제품들을 포함한다. 특히 RTX 5090은 초당 1.8TB의 데이터 전송량을 자랑하며, 배터리 효율도 개선된 것으로 밝혀졌다. 이는 인텔, 퀄컴, AMD 등과 치열한 경쟁이 예상되는 상황을 초래한다.assistant

엔비디아가 '지포스 RTX 50 시리즈'를 발표하며 PC용 AI칩 시장을 겨냥한다. 이 시리즈는 '블랙웰' 아키텍처를 탑재한 새로운 GPU로, AI 기술과 가격이 강화된 제품들을 포함한다. 특히 RTX 5090은 초당 1.8TB의 데이터 전송량을 자랑하며, 배터리 효율도 개선된 것으로 밝혀졌다. 이는 인텔, 퀄컴, AMD 등과 치열한 경


In [20]:
# 주관식 문제 생성 지시
instruction = "다음 요약 내용을 기반으로 문제와 정답을 생성해주세요.\n\n"

messages = [
    {"role": "user", "content": f"{instruction}{summary}"}
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

# 텍스트 생성
outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [21]:
# 생성된 문제와 정답 출력
qa_pair = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
print("생성된 주관식 문제와 정답:")
print(qa_pair)

생성된 주관식 문제와 정답:
단답형 문제와 정답을 생성해 드리겠습니다.

1. 엔비디아가 '지포스 RTX 50 시리즈'를 발표하며 PC용 AI칩 시장을 겨냥하는 이유는?
   - 정답: 엔비디아가 '지포스 RTX 50 시리즈'를 발표하며 PC용 AI칩 시장을 겨냥하는 이유는 '블랙웰' 아키텍처를 탑재한 새로운 GPU와 AI 기술, 가격 강화된 제품들을 포함하는 것입니다.

2. RTX 5090의 데이터 전송량은 무엇인가요?
   - 정답: RTX 5090의 데이터 전송량은 초당 1.8TB입니다.

3. RTX 5090의 배터리 효율은 무엇인가요?
   - 정답: RTX 5090의 배터리 효율도 개선된 것으로 밝혀졌습니다.

4. 엔비디아의 '지포스 RTX 50 시리즈'가 인텔, 퀄컴, AMD 등과 치열한 경쟁을 초래하는 이유는 무엇인가요?
   - 정답: 엔비디아의
